In [1]:
from sympy import Matrix, Symbol, derive_by_array, Lambda, symbols, Derivative, diff
from sympy.abc import x, y, i, j, a, b


Defining variable-element matrices $X \in \mathbb{R}^{n \times m}$ and $W \in \mathbb{R}^{m \times p}$:

In [2]:
def var(letter: str, i: int, j: int) -> Symbol:
    letter_ij = Symbol('{}_{}{}'.format(letter, i+1, j+1), is_commutative=True)
    return letter_ij


n,m,p = 3,3,2

X = Matrix(n, m, lambda i,j : var('x', i, j)); X

Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]])

In [3]:
W = Matrix(m, p, lambda i,j : var('w', i, j)); W

Matrix([
[w_11, w_12],
[w_21, w_22],
[w_31, w_32]])

Defining $N = \nu(X, W) = X \times W$

* $\nu : \mathbb{R}^{(n \times m) \times (m \times p)} \rightarrow \mathbb{R}^{n \times p}$
* $N \in \mathbb{R}^{n \times p}$

In [4]:
v = Lambda((a,b), a*b); v

Lambda((a, b), a*b)

In [5]:
N = v(X, W); N

Matrix([
[w_11*x_11 + w_21*x_12 + w_31*x_13, w_12*x_11 + w_22*x_12 + w_32*x_13],
[w_11*x_21 + w_21*x_22 + w_31*x_23, w_12*x_21 + w_22*x_22 + w_32*x_23],
[w_11*x_31 + w_21*x_32 + w_31*x_33, w_12*x_31 + w_22*x_32 + w_32*x_33]])


Defining $S = \sigma_{\text{apply}}(N) = \sigma_{\text{apply}}(\nu(X,W)) = \sigma_\text{apply}(X \times W) = \Big \{ \sigma(XW_{ij}) \Big\}$.


Assume that $\sigma_{\text{apply}} : \mathbb{R}^{n \times p} \rightarrow \mathbb{R}^{n \times p}$ while $\sigma : \mathbb{R} \rightarrow \mathbb{R}$, so the function $\sigma_{\text{apply}}$ takes in a matrix and returns a matrix while the simple $\sigma$ acts on the individual elements $N_{ij} = XW_{ij}$ in the matrix argument $N$ of $\sigma_{\text{apply}}$.

* $\sigma : \mathbb{R} \rightarrow \mathbb{R}$
* $\sigma_\text{apply} : \mathbb{R}^{n \times p} \rightarrow \mathbb{R}^{n \times p}$
* $S \in \mathbb{R}^{n \times p}$

In [6]:
from sympy import Function

# Nvec = Symbol('N', commutative=False)

sigma = Function('sigma')
sigma(N[0,0])

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13)

In [7]:
# way 1 of declaring S
S = N.applyfunc(sigma); S
#type(S)
#Matrix(3, 2, lambda i, j: sigma(N[i,j]))

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [8]:
# way 2 of declaring S (better way)
sigmaApply = lambda matrix:  matrix.applyfunc(sigma)

sigmaApply(N)

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [9]:
sigmaApply(X**2) # can apply this function to any matrix argument.

Matrix([
[  sigma(x_11**2 + x_12*x_21 + x_13*x_31), sigma(x_11*x_12 + x_12*x_22 + x_13*x_32), sigma(x_11*x_13 + x_12*x_23 + x_13*x_33)],
[sigma(x_11*x_21 + x_21*x_22 + x_23*x_31),   sigma(x_12*x_21 + x_22**2 + x_23*x_32), sigma(x_13*x_21 + x_22*x_23 + x_23*x_33)],
[sigma(x_11*x_31 + x_21*x_32 + x_31*x_33), sigma(x_12*x_31 + x_22*x_32 + x_32*x_33),   sigma(x_13*x_31 + x_23*x_32 + x_33**2)]])

In [10]:
S = sigmaApply(v(X,W)) # composing
S

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

Defining $L = \Lambda(S) = \Lambda(\sigma_\text{apply}(\nu(X,W))) = \Lambda \Big(\Big \{ \sigma(XW_{ij}) \Big\} \Big)$. In general, let the function be defined as:
$$
\begin{align}
L &= \Lambda \begin{pmatrix}
   \sigma(XW_{11}) & \sigma(XW_{12}) & ... & \sigma(XW_{1p}) \\
   \sigma(XW_{21}) & \sigma(XW_{22}) & ... & \sigma(XW_{2p}) \\
   \vdots & \vdots & & \vdots \\
   \sigma(XW_{n1}) & \sigma(XW_{n2}) & ... & \sigma(XW_{np})
\end{pmatrix} \\

&= \sum_{i=1}^p \sum_{j = 1}^n  \sigma(XW_{ij}) \\

&= \sigma(XW_{11}) + \sigma{XW_{12}} + ... + \sigma(XW_{np})
\end{align}
$$
* $\Lambda: \mathbb{R}^{n \times p} \rightarrow \mathbb{R}$
* $L \in \mathbb{R}$

In [11]:
lambdaF = lambda matrix : sum(matrix)
lambdaF(S)

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13) + sigma(w_11*x_21 + w_21*x_22 + w_31*x_23) + sigma(w_11*x_31 + w_21*x_32 + w_31*x_33) + sigma(w_12*x_11 + w_22*x_12 + w_32*x_13) + sigma(w_12*x_21 + w_22*x_22 + w_32*x_23) + sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)

In [12]:
L = lambdaF(sigmaApply(v(X, W)))
L
#L = lambda mat1, mat2: lambdaF(sigmaApply(v(mat1, mat2)))
#L(X, W)

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13) + sigma(w_11*x_21 + w_21*x_22 + w_31*x_23) + sigma(w_11*x_31 + w_21*x_32 + w_31*x_33) + sigma(w_12*x_11 + w_22*x_12 + w_32*x_13) + sigma(w_12*x_21 + w_22*x_22 + w_32*x_23) + sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)

In [13]:
#derive_by_array(L, X)

In [14]:
derive_by_array(L, S)

[[1, 1], [1, 1], [1, 1]]

In [15]:
from sympy import sympify, lambdify
n = lambdify((X[0,0],X[0,1],X[0,2],W[0,0],W[1,0],W[2,0]), N[0,0])
n(1,2,3,4,3,2)

16

In [16]:
f = Function('f') #(sympify(N[0,0]))
f(N[0,0])

f(w_11*x_11 + w_21*x_12 + w_31*x_13)

In [17]:
f(N[0,0]).diff(X[0,0])




w_11*Subs(Derivative(f(_xi_1), _xi_1), _xi_1, w_11*x_11 + w_21*x_12 + w_31*x_13)

In [18]:
n = v(X,W); n
n11 = Function('{}'.format(n[0,0]))
n11

w_11*x_11 + w_21*x_12 + w_31*x_13

In [19]:
s_ij = Function('s_ij')
sig = Function('sig')(x)

In [20]:

# KEY: got not expecting UndefinedFunction error again here too
#S_ij = Matrix(3, 2, lambda i,j: Function('s_{}{}'.format(i+1,j+1))(Function('{}'.format(N[i,j]))))



In [21]:
#S_ij[0,0](sympify(N[0,0])).diff(sympify(N[0,0]))
F = 3*x*y

xy = Symbol('{}'.format(F))
xy.subs({x:3})
sympify(xy).subs({x:3})

3*x*y

Sympy Example of trying to differentiate with respect to an **expression** not just a variable.

In [22]:
from sympy.abc import t

F = Function('F')
f = Function('f')
U = f(t)
V = U.diff(t)

direct = F(t, U, V).diff(U); direct

Subs(Derivative(F(t, _xi_2, Derivative(f(t), t)), _xi_2), _xi_2, f(t))

In [23]:
F(t,U,V)

F(t, f(t), Derivative(f(t), t))

In [24]:
F(t,U,V).subs(U,x)

F(t, x, Derivative(x, t))

In [25]:
F(t,U,V).subs(U,x).diff(x)

Subs(Derivative(F(t, _xi_2, Derivative(x, t)), _xi_2), _xi_2, x)

In [26]:
F(t,U,V).subs(U,x).diff(x).subs(x, U)

Subs(Derivative(F(t, _xi_2, Derivative(f(t), t)), _xi_2), _xi_2, f(t))

In [27]:
indirect = F(t,U,V).subs(U, x).diff(x).subs(x,U); indirect

Subs(Derivative(F(t, _xi_2, Derivative(f(t), t)), _xi_2), _xi_2, f(t))

In [28]:
F = Lambda((x,y), 3*x* y)
F(1,2)

6

In [29]:
U = x*y
G = 3*x*y
xy

3*x*y

In [30]:
F.diff(xy)

0

In [31]:
# derive_by_array(S, N) # ERROR

In [32]:
s11 = S[0,0]
s11

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13)

In [33]:

#s11.diff(n11)

In [34]:
derive_by_array(L, S)

[[1, 1], [1, 1], [1, 1]]

In [35]:

x, y, r, t = symbols('x y r t') # r (radius), t (angle theta)
f, g, h = symbols('f g h', cls=Function)
h = g(f(x))
Derivative(h, f(x)).doit()




Derivative(g(f(x)), f(x))

In [36]:
h.args[0]
h.diff(h.args[0])

Derivative(g(f(x)), f(x))

In [37]:
S = sigmaApply(v(X,W)); S

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [38]:
from sympy.abc import n

n11 = (X*W)[0,0]
m = lambda mat1, mat2: sympify(Symbol('{}'.format((mat1 * mat2)[0,0] )))
s = sigma(m(X,W)); s

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13)

In [39]:
s.subs({W[0,0]: 14}) # doesn't work to substitute into an undefined function

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13)

In [40]:
Derivative(s, m(X,W)).doit()

Derivative(sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), w_11*x_11 + w_21*x_12 + w_31*x_13)

In [41]:

#s11 = Function('s_{11}')(n11); s11
#sigma(n11).diff(n11)

#s11.diff(n11)
sigma(n11)

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13)

In [42]:
# ERROR HERE TOO
type(sigma(n11).args[0])

sympy.core.add.Add

In [43]:
type(n11)

sympy.core.add.Add

In [44]:
#sigma(n11).diff(sigma(n11).args[0]) ## ERROR

In [45]:
b = Symbol('{}'.format(n11))
ns_11 = Function(b, real=True)
ns_11


# ERROR cannot diff wi.r. to undefinedfunction
# sigma(n11).diff(ns_11)


#
#sigma(b).diff(b).subs({b:1})

w_11*x_11 + w_21*x_12 + w_31*x_13

In [46]:
f, g = symbols('f g', cls=Function)
xy = Symbol('x*y'); xy
#sympify(xy).subs({x:2, y:4})
f(g(x,y)).diff(xy)

0

In [47]:
# TODO SEEM to have got the expression but it is not working since can't substitute anything .... ???
f(xy).diff(xy).subs({x:2})

Derivative(f(x*y), x*y)

In [48]:
Function("x*y")(x,y)
xyf = lambdify([x,y],xy)
xyf(3,4)
f(g(xy)).diff(xy)
#

Derivative(f(g(x*y)), g(x*y))*Derivative(g(x*y), x*y)

In [49]:
xyd = Derivative(x*y, x*y,0).doit();xyd

#Derivative(3*xyd, xyd, 1).doit() ### ERROR can't calc deriv w.r.t to x*y

x*y

In [50]:
#derive_by_array(S, N)



